In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Pulling data from NASDAQ
Start with:
### ListMarketCenters
- Reference: http://www.nasdaqdod.com/NASDAQAnalytics.asmx?op=ListMarketCenters
- HTTP GET: http://ws.nasdaqdod.com/v1/NASDAQAnalytics.asmx/ListMarketCenters?%20HTTP/1.1

In [45]:
from lxml import objectify
from urllib import urlopen

path = 'http://ws.nasdaqdod.com/v1/NASDAQAnalytics.asmx/ListMarketCenters?%20HTTP/1.1'
xml = objectify.parse(urlopen(path))

columns = ['Outcome', 'Identity', 'Identity', 'Symbol', 'Code', 'Name']
root = xml.getroot()
print columns
print len(root.getchildren())

l = [ line.getchildren() for line in root.getchildren() ]
lp = [ line.getchildren() for line in root.getchildren() ]

df = pd.DataFrame(data=l, columns=columns)

print lp



['Outcome', 'Identity', 'Identity', 'Symbol', 'Code', 'Name']
19
[['Success', 'Request', 0, 'NSDQ', 'Q', 'The NASDAQ Stock Market LLC'], ['Success', 0, 'CBOE', 'W', 'CBOE Stock Exchange'], ['Success', 0, 'BX', 'B', 'NASDAQ OMX BX, Inc'], ['Success', 0, 'CHXE', 'M', 'Chicago Stock Exchange'], ['Success', 0, 'CINN', 'C', 'National Stock Exchange'], ['Success', 0, 'ISEG', 'I', 'International Securities Exchange'], ['Success', 0, 'ADFN', 'D', 'Industry Regulatory Authority'], ['Success', 0, 'BATS', 'Z', 'BATS Exchange Inc.'], ['Success', 0, 'ARCX', 'P', 'NYSE Arca'], ['Success', 0, 'NYSE', 'N', 'NYSE Euronext'], ['Success', 0, 'AMEX', 'A', 'NYSE AMEX'], ['Success', 0, 'INDP', 'E', 'Market Independent'], ['Success', 0, 'EDGA', 'J', 'EDGA Exchange Inc.'], ['Success', 0, 'EDGX', 'K', 'EDGX Exchange Inc.'], ['Success', 0, 'PHLX', 'X', 'NASDAQ OMX PSX'], ['Success', 0, 'BATSY', 'Y', 'BATS Y-Exchange Inc.'], ['Success', 0, 'NSDQT', 'T', 'The NASDAQ Stock Market LLC'], ['Success', 0, 'OTCBB', 'U'

In [85]:
import xml.etree.ElementTree as ET

tree = ET.parse(urlopen(path))
root1 = tree.getroot()

d = {}

for name in root1[0]:
    d[name.tag.split('}')[1]] = []

for root in root1:
    for child in root:
        d[child.tag.split('}')[1]].append(child.text)

In [90]:
print d.keys()
print d['Symbol']
print d['Code']

['Code', 'Name', 'Symbol', 'Delay', 'Outcome', 'Identity']
['NSDQ', 'CBOE', 'BX', 'CHXE', 'CINN', 'ISEG', 'ADFN', 'BATS', 'ARCX', 'NYSE', 'AMEX', 'INDP', 'EDGA', 'EDGX', 'PHLX', 'BATSY', 'NSDQT', 'OTCBB', 'NNOTC']
['Q', 'W', 'B', 'M', 'C', 'I', 'D', 'Z', 'P', 'N', 'A', 'E', 'J', 'K', 'X', 'Y', 'T', 'U', 'u']


### HttpGet Request for GetSummarizedTrades
- Reference: http://www.nasdaqdod.com/NASDAQAnalytics.asmx?op=GetSummarizedTrades
- HTTP GET: http://ws.nasdaqdod.com/v1/NASDAQAnalytics.asmx/GetSummarizedTrades?Symbols=string&StartDateTime=string&EndDateTime=string&MarketCenters=string&TradePrecision=string&TradePeriod=string HTTP/1.1

In [104]:
path1 = 'http://ws.nasdaqdod.com/v1/NASDAQAnalytics.asmx/GetSummarizedTrades?Symbols=NSDQ&StartDateTime=3/25/2016 00:00:00.000&EndDateTime=3/25/2016 20:00:00.000&MarketCenters=Q&TradePrecision=Hour&TradePeriod=60'
xml = objectify.parse(urlopen(path1))
root = xml.getroot()
r = root.getchildren()
r[0].getchildren()[0].text

'RegistrationError'

### Ending NASDAQ pull because I've made too many requests in one day...

# List of compnaies

In [4]:
companies = pd.read_csv('companylist.csv')

In [108]:
companies.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,Country,IPOyear,Sector,Industry,Summary Quote,Unnamed: 10
0,TFSC,1347 Capital Corp.,9.95,59182600.00,n/a,United States,2014,Finance,Business Services,http://www.nasdaq.com/symbol/tfsc,NaN
1,TFSCR,1347 Capital Corp.,0.32,0.00,n/a,United States,2014,Finance,Business Services,http://www.nasdaq.com/symbol/tfscr,NaN
2,TFSCU,1347 Capital Corp.,10.25,42845000.00,n/a,United States,2014,Finance,Business Services,http://www.nasdaq.com/symbol/tfscu,NaN
3,TFSCW,1347 Capital Corp.,0.22,0.00,n/a,United States,2014,Finance,Business Services,http://www.nasdaq.com/symbol/tfscw,NaN
4,PIH,"1347 Property Insurance Holdings, Inc.",6.19,37876145.75,n/a,United States,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN


In [111]:
len(companies)

6703

### Historical Stock Prices
Reference: http://www.nasdaq.com/symbol/pih/historical

In [126]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.nasdaq.com/symbol/pih/historical'
print 'making request to: {0}'.format(url)

content = urlopen(url).read()
soup = BeautifulSoup(content)
titles = soup.select('div#quotes_content_left_pnlAJAX table tbody tr td')


making request to: http://www.nasdaq.com/symbol/pih/historical


In [142]:
list_of_numbers = []
for row in titles:
    if row.text.strip() != '':
        list_of_numbers.append(row.text.strip())

formatted_list = []
for n in xrange(len(list_of_numbers)/6):
    formatted_list.append([list_of_numbers[n * 6], list_of_numbers[n * 6 + 1], list_of_numbers[n * 6 + 2], list_of_numbers[n * 6 + 3], list_of_numbers[n * 6 + 4], list_of_numbers[n * 6 + 5]])

In [143]:
formatted_list

[[u'03/24/2016', u'5.98', u'6.23', u'5.895', u'6.19', u'21,032'],
 [u'03/23/2016', u'5.9', u'5.96', u'5.82', u'5.95', u'14,802'],
 [u'03/22/2016', u'5.819', u'5.89', u'5.695', u'5.89', u'50,204'],
 [u'03/21/2016', u'5.602', u'5.8', u'5.602', u'5.7', u'20,932'],
 [u'03/18/2016', u'5.43', u'5.6888', u'5.421', u'5.56', u'31,567'],
 [u'03/17/2016', u'5.61', u'5.68', u'5.32', u'5.4', u'34,082'],
 [u'03/16/2016', u'5.58', u'5.61', u'5.58', u'5.61', u'663'],
 [u'03/15/2016', u'5.59', u'5.62', u'5.547', u'5.547', u'1,248'],
 [u'03/14/2016', u'5.55', u'5.6033', u'5.5161', u'5.53', u'4,988'],
 [u'03/11/2016', u'5.59', u'5.68', u'5.44', u'5.56', u'5,185'],
 [u'03/10/2016', u'5.7', u'5.7', u'5.38', u'5.6', u'36,932'],
 [u'03/09/2016', u'5.7458', u'5.86', u'5.7346', u'5.78', u'1,445'],
 [u'03/08/2016', u'5.86', u'5.86', u'5.7333', u'5.7439', u'3,605'],
 [u'03/07/2016', u'5.68', u'5.84', u'5.68', u'5.74', u'6,137'],
 [u'03/04/2016', u'5.59', u'5.7295', u'5.59', u'5.66', u'7,640'],
 [u'03/03/2016', u

In [150]:
list_of_urls = [ [com.strip(), 'http://www.nasdaq.com/symbol/{0}/historical'.format(com.strip())] for com in companies['Symbol'] ]

In [151]:
list_of_urls

[['TFSC', 'http://www.nasdaq.com/symbol/TFSC/historical'],
 ['TFSCR', 'http://www.nasdaq.com/symbol/TFSCR/historical'],
 ['TFSCU', 'http://www.nasdaq.com/symbol/TFSCU/historical'],
 ['TFSCW', 'http://www.nasdaq.com/symbol/TFSCW/historical'],
 ['PIH', 'http://www.nasdaq.com/symbol/PIH/historical'],
 ['FLWS', 'http://www.nasdaq.com/symbol/FLWS/historical'],
 ['FCTY', 'http://www.nasdaq.com/symbol/FCTY/historical'],
 ['FCCY', 'http://www.nasdaq.com/symbol/FCCY/historical'],
 ['SRCE', 'http://www.nasdaq.com/symbol/SRCE/historical'],
 ['VNET', 'http://www.nasdaq.com/symbol/VNET/historical'],
 ['XXII', 'http://www.nasdaq.com/symbol/XXII/historical'],
 ['TWOU', 'http://www.nasdaq.com/symbol/TWOU/historical'],
 ['DDD', 'http://www.nasdaq.com/symbol/DDD/historical'],
 ['MMM', 'http://www.nasdaq.com/symbol/MMM/historical'],
 ['WBAI', 'http://www.nasdaq.com/symbol/WBAI/historical'],
 ['JOBS', 'http://www.nasdaq.com/symbol/JOBS/historical'],
 ['WUBA', 'http://www.nasdaq.com/symbol/WUBA/historical'

In [145]:
'hi {0} there'.format('you')

'hi you there'

In [154]:
company_info = {}

for url in list_of_urls[:10]:
    content = urlopen(url[1]).read()
    soup = BeautifulSoup(content)
    titles = soup.select('div#quotes_content_left_pnlAJAX table tbody tr td')
    
    list_of_numbers = []
    for row in titles:
        if row.text.strip() != '':
            list_of_numbers.append(row.text.strip())

    formatted_list = []
    for n in xrange(len(list_of_numbers)/6):
        formatted_list.append([list_of_numbers[n * 6], list_of_numbers[n * 6 + 1], list_of_numbers[n * 6 + 2], list_of_numbers[n * 6 + 3], list_of_numbers[n * 6 + 4], list_of_numbers[n * 6 + 5]])
    
    company_info[url[0]] = formatted_list

In [155]:
company_info

{'FCCY': [[u'03/24/2016', u'12.55', u'12.9', u'12.55', u'12.9', u'4,801'],
  [u'03/23/2016', u'12.87', u'12.9', u'12.24', u'12.89', u'2,920'],
  [u'03/22/2016', u'12.72', u'12.96', u'12.6101', u'12.75', u'9,170'],
  [u'03/21/2016', u'12.58', u'12.81', u'12.31', u'12.72', u'3,958'],
  [u'03/18/2016', u'12.49', u'12.68', u'12', u'12.68', u'119,323'],
  [u'03/17/2016', u'12.6', u'12.8', u'12.1', u'12.18', u'21,929'],
  [u'03/16/2016', u'12.9', u'12.9', u'12.5', u'12.52', u'10,497'],
  [u'03/15/2016', u'12.99', u'12.99', u'12.52', u'12.95', u'8,751'],
  [u'03/14/2016', u'13.15', u'13.3', u'12.68', u'12.79', u'22,330'],
  [u'03/11/2016', u'12.79', u'13.24', u'12.79', u'13.03', u'19,985'],
  [u'03/10/2016', u'12.84', u'12.85', u'12.36', u'12.8', u'23,197'],
  [u'03/09/2016', u'12.6', u'12.95', u'12.45', u'12.7', u'9,524'],
  [u'03/08/2016', u'12.623', u'12.75', u'12.3', u'12.43', u'15,993'],
  [u'03/07/2016', u'11.91', u'12.96', u'11.85', u'12.74', u'35,609'],
  [u'03/04/2016', u'11.871', u'

In [159]:
company_info.keys()[0]

'PIH'

In [5]:
clean_names = list(companies['Name'])

In [6]:
clean_names[:5]

['1347 Capital Corp.',
 '1347 Capital Corp.',
 '1347 Capital Corp.',
 '1347 Capital Corp.',
 '1347 Property Insurance Holdings, Inc.']

In [ ]:
http://api.nytimes.com/svc/search/v2/articlesearch?q=apple
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=new+york+times&page=2&sort=oldest&api-key=198fa145ebe84d6acc131bcdfe80241f:13:74828369

In [7]:
ls

Scraping NASDAQ.ipynb  data/                  scrape_nasdaq.py
companylist.csv        scrap_update.py        test.csv


In [8]:
ls data

company_data.csv  fails.csv


In [23]:
df_coms = pd.read_csv('data/company_data.csv', )
df_coms['date'] = pd.to_datetime(df_coms['date']).apply(pd.datetools.normalize_date)
df_coms.head()

,Unnamed: 0,name,date,open,high,low,close,volume
0,0,TFSC,2016-03-18,9.810,9.810,9.810,9.810,100
1,1,TFSC,2016-03-17,9.869,9.869,9.869,9.869,00
2,2,TFSC,2016-03-16,9.869,9.869,9.869,9.869,00
3,3,TFSC,2016-03-15,9.869,9.869,9.869,9.869,00
4,4,TFSC,2016-03-14,9.869,9.869,9.869,9.869,00


In [27]:
len(df_coms)/6703

52

In [31]:
df_aapl = df_coms[df_coms['name'] == 'AAPL']
print len(df_aapl)
print df_aapl.head()

58
       Unnamed: 0  name       date    open    high     low   close      volume
23084       23084  AAPL 2016-03-18  106.34  106.50  105.19  105.92  43,607,120
23085       23085  AAPL 2016-03-17  105.52  106.47  104.96  105.80  34,394,830
23086       23086  AAPL 2016-03-16  104.61  106.31  104.59  105.97  38,236,730
23087       23087  AAPL 2016-03-15  103.96  105.18  103.85  104.58  40,026,250
23088       23088  AAPL 2016-03-14  101.91  102.91  101.78  102.52  25,058,280


In [47]:
mids = (df_aapl['open'] + df_aapl['close'])/2
spans = abs(df_aapl['open'] - df_aapl['close'])
inc = df_aapl['close'] > df_aapl['open']
dec = df_aapl['close'] < df_aapl['open']
w = 58


In [55]:
from math import pi

import pandas as pd

from bokeh.plotting import figure, show, output_file
# from bokeh.sampledata.stocks import MSFT
from bokeh.io import output_notebook

from bokeh.models.widgets import TextInput
from bokeh.io import vform

text_input = TextInput(value="AAPL", title="Label:")
df_aapl = df_coms[df_coms['name'] == text_input.value]
mids = (df_aapl['open'] + df_aapl['close'])/2
spans = abs(df_aapl['open'] - df_aapl['close'])
inc = df_aapl['close'] > df_aapl['open']
dec = df_aapl['close'] < df_aapl['open']

# mids = (df.open + df.close)/2
# spans = abs(df.close-df.open)

# inc = df.close > df.open
# dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left")

p.title = "MSFT Candlestick"
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

p.segment(df_aapl.date, df_aapl.high, df_aapl.date, df_aapl.low, color="black")
p.rect(df_aapl.date[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
p.rect(df_aapl.date[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")

output_file("candlestick.html", title="candlestick.py example")
output_notebook()
show(p)  # open a browser

Loading BokehJS ...

In [50]:
from bokeh.models.widgets import TextInput
from bokeh.io import output_file, show, vform

output_file("text_input.html")

text_input = TextInput(value="default", title="Label:")

show(vform(text_input))

In [51]:
text_input.value

'default'

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib2 import urlopen

from math import pi
from bokeh.plotting import figure, show, output_file
# from bokeh.sampledata.stocks import MSFT
from bokeh.io import output_notebook

from bokeh.models.widgets import TextInput
from bokeh.io import vform

from flask import Flask, render_template, request
app = Flask(__name__)

# ---------- DEFINE FUNCTIONS ----------

def scrape_page(sym):
    url = 'http://www.nasdaq.com/symbol/{0}/historical'.format(sym.strip())
    try:
        company_info = []
        content = urlopen(url).read()
        soup = BeautifulSoup(content)
        titles = soup.select('div#quotes_content_left_pnlAJAX table tbody tr td')

        list_of_numbers = []
        for row in titles:  # parses all the values returned from beautiful soup pull
            if row.text.strip() != '':
                list_of_numbers.append(row.text.strip())

        formatted_list = []
        # Need to skip the first row of data becuase it is not useful...
        for n in xrange(5, len(list_of_numbers)/6):   # creates an n by 6 list for each company
            #formatted_list.append([list_of_numbers[n * 6], list_of_numbers[n * 6 + 1], list_of_numbers[n * 6 + 2], list_of_numbers[n * 6 + 3], list_of_numbers[n * 6 + 4], list_of_numbers[n * 6 + 5]])
            company_info.append([sym, int(list_of_numbers[n * 6]), int(list_of_numbers[n * 6 + 1]), int(list_of_numbers[n * 6 + 2]), int(list_of_numbers[n * 6 + 3]), int(list_of_numbers[n * 6 + 4]), int(list_of_numbers[n * 6 + 5])])

        return company_info
    except:
        return "Could not find company"

def render_table(sym):
    company_info = scrape_page(sym)
    columns = ['name', 'date', 'open', 'high', 'low', 'close', 'volume']
    df = pd.DataFrame(company_info, columns=columns)
    return df
    df['date'] = pd.to_datetime(df['date']).apply(pd.datetools.normalize_date)

    text_input = TextInput(value=sym.upper(), title="Label:")
    mids = (df['open'] + df['close'])/2
    spans = abs(df['open'] - df['close'])
    inc = df['close'] > df['open']
    dec = df['close'] < df['open']
    w = 12*60*60*1000 # half day in ms

    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

    p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, toolbar_location="left")

    p.title = "MSFT Candlestick"
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3

    p.segment(df.date, df.high, df.date, df.low, color="black")
    p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
    p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")

    output_file("templates/candlestick.html", title="candlestick.py example")

    return "candlestick.html"

In [7]:
df = render_table('AAPL')

In [20]:
company_info = scrape_page('AAPL')
print company_info
columns = ['name', 'date', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(company_info, columns=columns)

Could not find company


PandasError: DataFrame constructor not properly called!

In [26]:
url = 'http://www.nasdaq.com/symbol/AAPL/historical'
company_info = []
content = urlopen(url).read()
soup = BeautifulSoup(content)
titles = soup.select('div#quotes_content_left_pnlAJAX table tbody tr td')

list_of_numbers = []
for row in titles:  # parses all the values returned from beautiful soup pull
    if row.text.strip() != '':
        list_of_numbers.append(row.text.strip())

formatted_list = []
# Need to skip the first row of data becuase it is not useful...
for n in xrange(5, len(list_of_numbers)/6):   # creates an n by 6 list for each company
    #formatted_list.append([list_of_numbers[n * 6], list_of_numbers[n * 6 + 1], list_of_numbers[n * 6 + 2], list_of_numbers[n * 6 + 3], list_of_numbers[n * 6 + 4], list_of_numbers[n * 6 + 5]])
    company_info.append(['aapl', list_of_numbers[n * 6], float(list_of_numbers[n * 6 + 1]), float(list_of_numbers[n * 6 + 2]), float(list_of_numbers[n * 6 + 3]), float(list_of_numbers[n * 6 + 4]), int(list_of_numbers[n * 6 + 5].replace(",",''))])

